lest redo the first 10 cells again

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

## Building a Collaborative Filtering System from Scratch

First, let's import the necessary libraries:

```python
from fastai.imports import *
from fastai.collab import *
from fastai.tabular.all import *
set_seed(42)
```

We'll use the MovieLens 100k dataset, which contains 100,000 movie ratings:

```python
path = untar_data(URLs.ML_100k)
path.ls()
```

```
(#23) [Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u.item'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u3.test'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u1.base'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u.info'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u2.test'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u5.test'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u.genre'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/ub.test'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/ua.base'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u.data'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/README'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u4.test'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u5.base'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/ub.base'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/ua.test'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u4.base'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u.user'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/allbut.pl'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u3.base'),Path('/Users/gaurav.adlakha/.fastai/data/ml-100k/u1.test')...]
```

Let's load the ratings data:

```python
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()
```

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>user</th>
      <th>movie</th>
      <th>rating</th>
      <th>timestamp</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>196</td>
      <td>242</td>
      <td>3</td>
      <td>881250949</td>
    </tr>
    <tr>
      <th>1</th>
      <td>186</td>
      <td>302</td>
      <td>3</td>
      <td>891717742</td>
    </tr>
    <tr>
      <th>2</th>
      <td>22</td>
      <td>377</td>
      <td>1</td>
      <td>878887116</td>
    </tr>
    <tr>
      <th>3</th>
      <td>244</td>
      <td>51</td>
      <td>2</td>
      <td>880606923</td>
    </tr>
    <tr>
      <th>4</th>
      <td>166</td>
      <td>346</td>
      <td>1</td>
      <td>886397596</td>
    </tr>
  </tbody>
</table>
</div>

## Visualizing User-Movie Ratings

Let's create a cross-tabulation to see how specific users rated specific movies:

```python
def get_top_ratings(df, n_users=20, n_movies=20):
    "Return crosstab of ratings from top users for top movies"
    top_users = df.user.value_counts().index[:n_users]
    top_movies = df.movie.value_counts().index[:n_movies]
    filtered = df[(df.user.isin(top_users)) & (df.movie.isin(top_movies))]
    return pd.crosstab(filtered.user, filtered.movie, filtered.rating, aggfunc='mean')

ratings_matrix = get_top_ratings(ratings)
```

```python
ratings_matrix
```

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>movie</th>
      <th>1</th>
      <th>7</th>
      <th>50</th>
      <th>56</th>
      <th>98</th>
      <th>100</th>
      <th>117</th>
      <th>121</th>
      <th>127</th>
      <th>172</th>
      <th>174</th>
      <th>181</th>
      <th>204</th>
      <th>222</th>
      <th>237</th>
      <th>258</th>
      <th>286</th>
      <th>288</th>
      <th>294</th>
      <th>300</th>
    </tr>
    <tr>
      <th>user</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>7</th>
      <td>NaN</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>4.0</td>
      <td>5.0</td>
      <td>NaN</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>4.0</td>
      <td>5.0</td>
      <td>3.0</td>
      <td>5.0</td>
      <td>NaN</td>
      <td>5.0</td>
      <td>4.0</td>
      <td>4.0</td>
      <td>4.0</td>
      <td>1.0</td>
      <td>4.0</td>
    </tr>
    <tr>
      <th>13</th>
      <td>3.0</td>
      <td>2.0</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>4.0</td>
      <td>5.0</td>
      <td>3.0</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>4.0</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>3.0</td>
      <td>5.0</td>
      <td>4.0</td>
      <td>3.0</td>
      <td>1.0</td>
      <td>2.0</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>92</th>
      <td>4.0</td>
      <td>4.0</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>5.0</td>
      <td>4.0</td>
      <td>5.0</td>
      <td>NaN</td>
      <td>4.0</td>
      <td>5.0</td>
      <td>4.0</td>
      <td>4.0</td>
      <td>4.0</td>
      <td>4.0</td>
      <td>4.0</td>
      <td>NaN</td>
      <td>3.0</td>
      <td>3.0</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>
</div>

## The Math Behind Collaborative Filtering

We can represent users and movies as vectors in a latent feature space:

```python
movie = np.array([0.98, 0.9, -0.9])
user = np.array([0.9, 0.8, -0.6])
(user*movie).sum()
```

```
np.float64(2.1420000000000003)
```

The positive dot product (2.142) suggests this user would enjoy this movie. Each dimension represents a latent feature (e.g., action, drama, romance) that the algorithm learns automatically.

In [ ]:
cells_11_20= nb.cells[11:20]

lest redo the next 10 cell $`cells_11_20` it okay to add more info if its adding values 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

## Understanding User Preferences with Dot Products

Let's see how a different user might feel about the same movie:

```python
movie = np.array([0.98, 0.9, -0.9])
user = np.array([0.1, -1.0, -0.6])
(user*movie).sum()
```

```
np.float64(-0.262)
```

The negative dot product (-0.262) suggests this user would probably dislike this movie. Notice how this user has a negative value in the second dimension, opposing the movie's positive value there - this mismatch contributes to the negative overall score.

## Adding Movie Titles to Our Dataset

To make our analysis more interpretable, let's add movie titles to our dataset:

```python
movies = pd.read_csv(path/'u.item', delimiter='|', header=None, encoding='latin1', 
                    usecols=[0,1], names=['movie','title'])
movies.head()
```

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>movie</th>
      <th>title</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>Toy Story (1995)</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>GoldenEye (1995)</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>Four Rooms (1995)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>Get Shorty (1995)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5</td>
      <td>Copycat (1995)</td>
    </tr>
  </tbody>
</table>
</div>

Now we can see our full ratings dataset:

```python
ratings
```

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>user</th>
      <th>movie</th>
      <th>rating</th>
      <th>timestamp</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>196</td>
      <td>242</td>
      <td>3</td>
      <td>881250949</td>
    </tr>
    <tr>
      <th>1</th>
      <td>186</td>
      <td>302</td>
      <td>3</td>
      <td>891717742</td>
    </tr>
    <tr>
      <th>2</th>
      <td>22</td>
      <td>377</td>
      <td>1</td>
      <td>878887116</td>
    </tr>
    <tr>
      <th>3</th>
      <td>244</td>
      <td>51</td>
      <td>2</td>
      <td>880606923</td>
    </tr>
    <tr>
      <th>4</th>
      <td>166</td>
      <td>346</td>
      <td>1</td>
      <td>886397596</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>99995</th>
      <td>880</td>
      <td>476</td>
      <td>3</td>
      <td>880175444</td>
    </tr>
    <tr>
      <th>99996</th>
      <td>716</td>
      <td>204</td>
      <td>5</td>
      <td>879795543</td>
    </tr>
    <tr>
      <th>99997</th>
      <td>276</td>
      <td>1090</td>
      <td>1</td>
      <td>874795795</td>
    </tr>
    <tr>
      <th>99998</th>
      <td>13</td>
      <td>225</td>
      <td>2</td>
      <td>882399156</td>
    </tr>
    <tr>
      <th>99999</th>
      <td>12</td>
      <td>203</td>
      <td>3</td>
      <td>879959583</td>
    </tr>
  </tbody>
</table>
<p>100000 rows × 4 columns</p>
</div>

Let's merge the movie titles into our ratings dataframe:

```python
ratings = ratings.merge(movies)
ratings
```

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>user</th>
      <th>movie</th>
      <th>rating</th>
      <th>timestamp</th>
      <th>title</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>196</td>
      <td>242</td>
      <td>3</td>
      <td>881250949</td>
      <td>Kolya (1996)</td>
    </tr>
    <tr>
      <th>1</th>
      <td>186</td>
      <td>302</td>
      <td>3</td>
      <td>891717742</td>
      <td>L.A. Confidential (1997)</td>
    </tr>
    <tr>
      <th>2</th>
      <td>22</td>
      <td>377</td>
      <td>1</td>
      <td>878887116</td>
      <td>Heavyweights (1994)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>244</td>
      <td>51</td>
      <td>2</td>
      <td>880606923</td>
      <td>Legends of the Fall (1994)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>166</td>
      <td>346</td>
      <td>1</td>
      <td>886397596</td>
      <td>Jackie Brown (1997)</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>99995</th>
      <td>880</td>
      <td>476</td>
      <td>3</td>
      <td>880175444</td>
      <td>First Wives Club, The (1996)</td>
    </tr>
    <tr>
      <th>99996</th>
      <td>716</td>
      <td>204</td>
      <td>5</td>
      <td>879795543</td>
      <td>Back to the Future (1985)</td>
    </tr>
    <tr>
      <th>99997</th>
      <td>276</td>
      <td>1090</td>
      <td>1</td>
      <td>874795795</td>
      <td>Sliver (1993)</td>
    </tr>
    <tr>
      <th>99998</th>
      <td>13</td>
      <td>225</td>
      <td>2</td>
      <td>882399156</td>
      <td>101 Dalmatians (1996)</td>
    </tr>
    <tr>
      <th>99999</th>
      <td>12</td>
      <td>203</td>
      <td>3</td>
      <td>879959583</td>
      <td>Unforgiven (1992)</td>
    </tr>
  </tbody>
</table>
<p>100000 rows × 5 columns</p>
</div>

Now we can see that user 196 gave "Kolya (1996)" a rating of 3, and user 716 gave "Back to the Future (1985)" a perfect 5.

## Building the Collaborative Filtering Model

Let's try to create a DataLoader using fastai's collaborative filtering tools:

```python
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()
```

Once the DataLoader is created, we would access information about our dataset:

```python
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_movies
```


In [ ]:
cells_20_30= nb.cells[20:30]

lest redo the next 10 cell $`cells_20_30` always render the output cell as well like previous steps..


##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

## Creating and Exploring Embedding Factors

Once we have our DataLoader set up, let's examine the dataset dimensions:

```python
n_users
```

```
944
```

We have 944 unique users in our dataset. Now let's create random embedding factors for both users and movies:

```python
user_factor = torch.randn(n_users, 5)
movie_factor = torch.randn(n_movies, 5)
```

These are randomly initialized tensors that will serve as our latent factors. Each user and movie is represented by a 5-dimensional vector.

Let's check the shapes of these tensors:

```python
user_factor.shape
```

```
torch.Size([944, 5])
```

```python
movie_factor.shape
```

```
torch.Size([1665, 5])
```

This confirms we have 944 users and 1665 movies, each represented by 5 latent factors.

Let's look at a specific user's embedding:

```python
torch.embedding(user_factor, torch.tensor(5))
```

```
tensor([-1.2018, -1.2946, -1.8869,  1.2259,  0.2970])
```

Instead of manually indexing, we can use PyTorch's Embedding layer, which is designed for this purpose:

```python
user_emb = nn.Embedding(n_users, 5)
```

Now we can access user embeddings with a cleaner syntax:

```python
user_emb.weight[10]
```

```
tensor([-1.0398, -1.7286, -0.6798,  2.5864, -0.1015],
       grad_fn=<SelectBackward0>)
```

Let's compare with our original random factors:

```python
user_factor
```

```
tensor([[-1.0827,  0.2138,  0.9310, -0.2739, -0.4359],
        [-0.5195,  0.7613, -0.4365,  0.1365,  1.3300],
        [-1.2804,  0.0705,  0.6489, -1.2110,  1.8266],
        ...,
        [ 0.8009, -0.4734, -0.8962, -0.7348, -0.0246],
        [ 0.3354, -0.8262, -0.1541,  0.4699,  0.4873],
        [ 2.4054, -0.2156, -1.4126, -0.2467,  1.0571]])
```

And check a specific user:

```python
user_factor[10]
```

```
tensor([-0.5753,  0.1556, -0.3694,  0.4986, -2.5438])
```

The `nn.Embedding` layer is initialized with different random values than our manual initialization:

```python
user_emb
```

```
Embedding(944, 5)
```

These embedding layers are the core of collaborative filtering models. During training, these initially random values will be updated to reflect the patterns in the rating data. Users with similar tastes will end up with similar embedding vectors, and movies with similar appeal will likewise have similar embeddings.

The dot product between a user's embedding and a movie's embedding will predict the rating that user would give to that movie. By training on known ratings, the model learns to generalize to predict unknown ratings.

In [ ]:
cells_30_40= nb.cells[30:40]

lets do the next 10 cell $`cells_30_40` always render the output cell 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

## Understanding Embedding Operations

Taking the dot product with a one-hot encoding of a vector is the same as looking up that vector at a particular index.

Let's explore this concept:

```python
one_hot(3, 100)
```

```
tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=torch.uint8)
```

This creates a one-hot vector where the 3rd position (0-indexed) is 1 and all other positions are 0. Let's convert it to float for matrix operations:

```python
one_hot(3, 100).float()
```

```
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
```

Now let's look at a specific embedding weight:

```python
user_emb.weight[10]
```

```
tensor([-1.0398, -1.7286, -0.6798,  2.5864, -0.1015],
       grad_fn=<SelectBackward0>)
```

And check the embedding layer:

```python
user_emb
```

```
Embedding(944, 5)
```

The weight matrix has shape:

```python
user_emb.weight.shape
```

```
torch.Size([944, 5])
```

Let's create a one-hot vector for user 10:

```python
one_hot(10, n_users).float().shape
```

```
torch.Size([944])
```

Now we can demonstrate that matrix multiplication with a one-hot vector is equivalent to selecting that row:

```python
user_emb.weight.t() @ one_hot(10, n_users).float()
```

```
tensor([-1.0398, -1.7286, -0.6798,  2.5864, -0.1015], grad_fn=<MvBackward0>)
```

This gives us the same result as directly indexing `user_emb.weight[10]`. This is the mathematical foundation for how embedding lookups work.

## What is Embedding?

An embedding is a learned mapping from discrete objects (like users or movies) to vectors of continuous numbers in a lower-dimensional space.

```python
u_e = Embedding(944, 5)
```

This creates an embedding layer for 944 users, each represented by a 5-dimensional vector. The embedding layer is essentially a lookup table where:
- Each row corresponds to a user (or item)
- Each column represents a learned latent feature
- During training, these values are updated to capture patterns in the data

Embeddings are powerful because they:
1. Convert sparse, high-dimensional one-hot encodings into dense, low-dimensional vectors
2. Learn meaningful relationships between entities
3. Allow us to measure similarity between entities using vector operations like dot products

In our collaborative filtering model, the dot product between a user embedding and a movie embedding produces a predicted rating.

In [ ]:
cells_40_50= nb.cells[40:50]

lets do the next 10 cell $`cells_40_50` always render the output cell

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

## Building the Collaborative Filtering Model

Let's examine the weights of our embedding layer:

```python
u_e.weight[10]
```

```
tensor([0.0063, 0.0057, 0.0053, 0.0052, 0.0039], grad_fn=<SelectBackward0>)
```

Now let's look at our data loaders:

```python
dls
```

```
<fastai.tabular.data.TabularDataLoaders at 0x7a1ca6e53650>
```

We can access the training data loader:

```python
dls[0]
```

```
<fastai.tabular.core.TabWeightedDL at 0x7a1d3812a410>
```

Let's confirm our data loaders again:

```python
dls
```

```
<fastai.tabular.data.TabularDataLoaders at 0x7a1ca6e53650>
```

Now let's get a batch of data:

```python
batch = dls.one_batch()
```

Let's check the dimensions of our dataset:

```python
944, 1665
```

```
(944, 1665)
```

This confirms we have 944 users and 1665 movies.

Let's examine the movie indices in our batch:

```python
batch[0][:,1]
```

```
tensor([1330,  899,  230, 1391,  334, 1133,  897,  466,  668,  102,  236, 1443,
         528,  320, 1247,  256,  769,  143,  271, 1397,  210, 1544, 1442,  529,
          17,  611, 1052,  485,  623, 1525,  938,  503, 1544,   65,  816, 1227,
          93,  499,  179, 1179,  588, 1019,  304,    5,  710,  457,  861, 1006,
         320,  578,  899,   62,  177,  279,  328, 1496,  570, 1252, 1216, 1402,
         884,  457,  738, 1121])
```

Let's create embeddings for users and movies and check their shapes:

```python
us_em = Embedding(944, 5)
mo_em = Embedding(1665, 5)

batch = dls.one_batch()

x = batch[0][:,0]
y = batch[0][:,1]

print(us_em(x).shape, mo_em(y).shape)
```

```
torch.Size([64, 5]) torch.Size([64, 5])
```

This shows that for a batch of 64 examples, we get 64 user embeddings and 64 movie embeddings, each with 5 dimensions.

Now let's create a complete collaborative filtering model:

```python
class CollabNN(nn.Module):
    "Simple collaborative filtering model with embeddings"
    def __init__(self, n_users, n_items, n_factors=5):
        super().__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.item_factors = nn.Embedding(n_items, n_factors)
        
    def forward(self, x):
        users, items = x[:,0], x[:,1]
        u_embs = self.user_factors(users)
        i_embs = self.item_factors(items)
        return (u_embs * i_embs).sum(dim=1)
```

This model:
1. Creates embedding layers for users and items
2. In the forward pass, extracts user and item indices from the input
3. Looks up the corresponding embeddings
4. Computes the element-wise product of user and item embeddings
5. Sums across the embedding dimensions to get a predicted rating

Finally, let's instantiate our model and prepare a batch for training:

```python
model = CollabNN(n_users, n_movies)
batch = dls.one_batch()
```

This collaborative filtering model is remarkably simple yet powerful. It learns to represent users and movies in a shared latent space where the dot product of vectors predicts ratings. During training, the model will adjust these embeddings to minimize prediction error on the training data.

In [ ]:
cells_50_60= nb.cells[50:60]

lets do the next 10 cell $`cells_50_60` always render the output cell

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

## Training and Evaluating the Model

Now let's use our model to make predictions on a batch of data:

```python
model(batch[0])
```

```
tensor([ 7.0029,  2.1804,  0.4648, -0.3704, -3.6570, -2.5748, -1.2354, -0.4618,
         0.8670, -0.7257, -1.3684,  0.6179,  0.6468,  2.9299,  1.4910, -0.4101,
         0.6326, -0.6567,  2.2093,  2.4966, -5.0130, -3.6183,  3.9021,  4.4451,
        -0.0432, -2.0280, -3.6852, -5.9757, -1.5701, -1.1312,  0.8875, -1.5192,
        -1.2604, -0.9187, -1.3469, -0.6555, -1.2011, -0.6149,  0.3042,  1.4095,
        -1.7217,  0.3008, -0.0148,  0.3080,  2.2792,  3.7195, -0.1592, -0.6061,
         1.7568, -0.7674,  0.2440, -0.9074, -1.0106, -3.1345,  0.0641,  1.2300,
         3.4579, -0.4415, -1.4399, -3.0345,  1.6182, -1.2363, -1.8696, -1.8537],
       grad_fn=<SumBackward1>)
```

Let's check the shape of our predictions:

```python
model(batch[0]).shape
```

```
torch.Size([64])
```

The model outputs a single prediction for each of the 64 examples in our batch.

Now let's create a learner to train our model:

```python
mdl = CollabNN(n_users, n_movies)
learner = Learner(dls, mdl, loss_func=MSELossFlat())
```

Let's train the model for 5 epochs using the one-cycle policy:

```python
learner.fit_one_cycle(5, 5e-3)
```

```
epoch     train_loss  valid_loss  time    
0         16.511957   16.503958   00:05     
1         12.656537   13.187530   00:05     
2         4.729345    5.090328    00:06     
3         2.651412    3.003635    00:05     
4         2.324853    2.789970    00:05     
```

Let's get a batch of data to make predictions on:

```python
x, y = dls.one_batch()
```

Now let's examine the training history:

Let's create a more readable DataFrame of our training history:

```python
# Get the recorded losses
losses = learner.recorder.values

# Get training and validation losses (first two columns of values)
train_losses = [x[0] for x in losses]
valid_losses = [x[1] for x in losses]

# Create a dataframe to display them
pd.DataFrame({
    'Epoch': range(1, len(train_losses)+1),
    'Training Loss': train_losses,
    'Validation Loss': valid_losses
})
```

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Epoch</th>
      <th>Training Loss</th>
      <th>Validation Loss</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>16.511957</td>
      <td>16.503958</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>12.656537</td>
      <td>13.187530</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>4.729345</td>
      <td>5.090328</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>2.651412</td>
      <td>3.003635</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5</td>
      <td>2.324853</td>
      <td>2.789970</td>
    </tr>
  </tbody>
</table>
</div>

We can see that both training and validation loss decrease significantly over the epochs, indicating that our model is learning patterns in the data.

Now let's improve our model by constraining the output range to be between 0 and 5.5 (since ratings are on a 1-5 scale):

```python
class CollabNN(nn.Module):
    "Simple collaborative filtering model with embeddings"
    def __init__(self, n_users, n_items, n_factors=5, y_range=(0, 5.5)):
        super().__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.item_factors = nn.Embedding(n_items, n_factors)
        self.y_range = y_range

        
    def forward(self, x):
        users, items = x[:,0], x[:,1]
        u_embs = self.user_factors(users)
        i_embs = self.item_factors(items)
        return sigmoid_range((u_embs * i_embs).sum(dim=1), *self.y_range)
```

Let's create a new model and learner with this improved architecture:

```python
mdl = CollabNN(n_users, n_movies)
learner = Learner(dls, mdl, loss_func=MSELossFlat())
```

And train it again:

```python
learner.fit_one_cycle(5, 5e-3)
```

```
epoch     train_loss  valid_loss  time    
0         3.651534    3.777338    00:06     
1         2.176786    2.387183    00:06     
2         1.485974    1.752936    00:06     
3         1.194254    1.400067    00:06     
4         1.213866    1.356161    00:06     
```

This model converges much faster and achieves a lower loss. The sigmoid_range function constrains the output to be within the specified range, which helps the model learn more effectively.

In [ ]:
cells_60_70= nb.cells[60:70]


lets do the next 10 cell $`cells_60_70` always render the output cell

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

## Further Model Analysis and Improvements

Let's take a look at the training results of our modified model:

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Epoch</th>
      <th>Training Loss</th>
      <th>Validation Loss</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>3.529655</td>
      <td>3.523691</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>1.782526</td>
      <td>1.922908</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>1.169784</td>
      <td>1.352898</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>1.037823</td>
      <td>1.212110</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5</td>
      <td>1.031219</td>
      <td>1.192636</td>
    </tr>
  </tbody>
</table>
</div>

Let's test the sigmoid_range function with different values:

```python
x = torch.tensor([1, 2, 3, 4, 5, 10, 15, 100, 5.5])
```

```python
sigmoid_range(x, *(0, 5.5))
```

```
tensor([4.0208, 4.8444, 5.2392, 5.4011, 5.4632, 5.4998, 5.5000, 5.5000, 5.4776])
```

This shows how the sigmoid_range function maps values to the range [0, 5.5]. Note that extreme values (like 100) get clamped to the maximum of 5.5.


```python
class ModifiedCollabNN(nn.Module):
    "Collaborative filtering model with embeddings and bias terms"
    def __init__(self, n_users, n_items, n_factors=5, y_range=(0,5.5)):
        super().__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.item_factors = nn.Embedding(n_items, n_factors)
        self.user_bias = nn.Embedding(n_users, 1)
        self.item_bias = nn.Embedding(n_items, 1)
        self.bias = nn.Parameter(torch.zeros(1))
        self.y_range = y_range
        
    def forward(self, x):
        users, items = x[:,0], x[:,1]
        u_embs = self.user_factors(users)
        i_embs = self.item_factors(items)
        u_bias = self.user_bias(users).squeeze()
        i_bias = self.item_bias(items).squeeze()
        dot = (u_embs * i_embs).sum(dim=1)
        return sigmoid_range(dot + u_bias + i_bias + self.bias, *self.y_range)
```

This enhanced model includes:
1. User and movie embeddings (as before)
2. User bias terms (some users rate everything higher/lower)
3. Movie bias terms (some movies are generally rated higher/lower)
4. Global bias term (overall average rating)

Let's create a new model and learner with this architecture:

```python
mdl = ModifiedCollabNN(n_users, n_movies)
learner = Learner(dls, mdl, loss_func=MSELossFlat())
```

Let's train this improved model:

```python
learner.fit_one_cycle(5, 5e-3)
```

```
epoch     train_loss  valid_loss  time    
0         3.529655    3.523691    00:06     
1         1.782526    1.922908    00:06     
2         1.169784    1.352898    00:06     
3         1.037823    1.212110    00:06     
4         1.031219    1.192636    00:06     
```

Let's check the final training results:

```python
losses = learner.recorder.values

# Get training and validation losses (first two columns of values)
train_losses = [x[0] for x in losses]
valid_losses = [x[1] for x in losses]

# Create a dataframe to display them
pd.DataFrame({
    'Epoch': range(1, len(train_losses)+1),
    'Training Loss': train_losses,
    'Validation Loss': valid_losses
})
```

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Epoch</th>
      <th>Training Loss</th>
      <th>Validation Loss</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>3.529655</td>
      <td>3.523691</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>1.782526</td>
      <td>1.922908</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>1.169784</td>
      <td>1.352898</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>1.037823</td>
      <td>1.212110</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5</td>
      <td>1.031219</td>
      <td>1.192636</td>
    </tr>
  </tbody>
</table>
</div>

Our modified model with bias terms achieves a validation loss of about 1.19, which is a significant improvement over the initial model. This demonstrates the effectiveness of including bias terms in collaborative filtering models.

The bias terms allow the model to capture general tendencies:
- Some users consistently rate movies higher or lower than average
- Some movies are generally rated better or worse than average
- There's an overall baseline rating in the system

By combining these bias terms with the dot product of user and movie embeddings, we get a more accurate and nuanced prediction system.